In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import LabelEncoder


# Zadanie 1:  

## Email Spam

 Masz dane o 12 emailach z informacją czy to spam czy nie:

 **Zadania do wykonania:**

**a) Ręczne obliczenia**
1. Oblicz prawdopodobieństwa a priori: P(Spam=TAK) i P(Spam=NIE)
2. Dla każdej cechy oblicz prawdopodobieństwa warunkowe
3. Przewidź klasę dla nowego emaila:
```
   Słowo_1 = 'darmowy'
   Słowo_2 = 'wygrana'  
   Wykrzyknik = 'TAK'
```

Oblicz prawdopodobieństwa dla obu klas (TAK lub NIE) i znormalizuj

**b) Implementacja w Python**

1. Zaimplementuj obliczenia z punktu a) w Python (bez sklearn)
2. Porównaj wyniki z ręcznymi obliczeniami

**c) Sklearn**

1. Użyj `CategoricalNB` z sklearn do wytrenowania modelu
2. Porównaj wyniki z własnymi obliczeniami
3. Wyjaśnij różnice (jeśli są)

In [9]:
data_spam = {
    'Słowo_1': ['darmowy', 'darmowy', 'spotkanie', 'raport', 'oferta', 'darmowy',
                'spotkanie', 'oferta', 'raport', 'darmowy', 'spotkanie', 'oferta'],
    'Słowo_2': ['wygrana', 'wygrana', 'jutro', 'kwartalny', 'specjalna', 'rabat',
                'dziś', 'limitowana', 'miesięczny', 'rabat', 'pilne', 'wyjątkowa'],
    'Wykrzyknik': ['TAK', 'TAK', 'NIE', 'NIE', 'TAK', 'TAK',
                   'NIE', 'TAK', 'NIE', 'TAK', 'NIE', 'TAK'],
    'Spam': ['TAK', 'TAK', 'NIE', 'NIE', 'NIE', 'TAK',
             'NIE', 'NIE', 'NIE', 'TAK', 'NIE', 'NIE']
}


# Zadanie 1: Email Spam

In [10]:

df = pd.DataFrame(data_spam)

# --- Punkt a) i b): Implementacja ręczna w Python ---

# 1. Prawdopodobieństwa a priori (Priors)
total = len(df)
p_spam_tak = len(df[df['Spam'] == 'TAK']) / total
p_spam_nie = len(df[df['Spam'] == 'NIE']) / total

print("--- Wyniki obliczeń ręcznych ---")
print(f"P(Spam=TAK) = {p_spam_tak:.4f}")
print(f"P(Spam=NIE) = {p_spam_nie:.4f}")

# Nowy email
nowy_email = {'Słowo_1': 'darmowy', 'Słowo_2': 'wygrana', 'Wykrzyknik': 'TAK'}

# Funkcja do obliczania wiarygodności (Likelihood)
def oblicz_likelihood(klasa):
    subset = df[df['Spam'] == klasa]
    prob = 1.0
    for cecha, wartosc in nowy_email.items():
        count = len(subset[subset[cecha] == wartosc])
        prob_warunkowe = count / len(subset)
        prob *= prob_warunkowe
    return prob

like_tak = oblicz_likelihood('TAK')
like_nie = oblicz_likelihood('NIE')

# Prawdopodobieństwa a posteriori (nieznormalizowane)
posterior_tak = p_spam_tak * like_tak
posterior_nie = p_spam_nie * like_nie

# Normalizacja
suma = posterior_tak + posterior_nie
wynik_tak = posterior_tak / suma if suma > 0 else 0
wynik_nie = posterior_nie / suma if suma > 0 else 0

print(f"\nPredykcja dla nowego emaila:")
print(f"P(Spam=TAK | X) = {wynik_tak:.4f}")
print(f"P(Spam=NIE | X) = {wynik_nie:.4f}")

# --- Punkt c): Sklearn ---

print("\n--- Wyniki Sklearn (CategoricalNB) ---")
enc = OrdinalEncoder()
X = enc.fit_transform(df[['Słowo_1', 'Słowo_2', 'Wykrzyknik']])
y = (df['Spam'] == 'TAK').astype(int)

# Trenowanie modelu
model = CategoricalNB(alpha=1.0)
model.fit(X, y)

# Predykcja - tworzymy DataFrame dla danych testowych, aby uniknąć ostrzeżenia
test_data = pd.DataFrame(
    [['darmowy', 'wygrana', 'TAK']],
    columns=['Słowo_1', 'Słowo_2', 'Wykrzyknik']
)

test_vector = enc.transform(test_data)
proba = model.predict_proba(test_vector)[0]

print(f"Sklearn P(Spam=NIE) = {proba[0]:.4f}")
print(f"Sklearn P(Spam=TAK) = {proba[1]:.4f}")

--- Wyniki obliczeń ręcznych ---
P(Spam=TAK) = 0.3333
P(Spam=NIE) = 0.6667

Predykcja dla nowego emaila:
P(Spam=TAK | X) = 1.0000
P(Spam=NIE | X) = 0.0000

--- Wyniki Sklearn (CategoricalNB) ---
Sklearn P(Spam=NIE) = 0.0321
Sklearn P(Spam=TAK) = 0.9679


# Zadanie 2

## Diagnoza grypy

Lekarz diagnozuje grypę na podstawie objawów u 15 pacjentów:

a) Analiza danych

1. Ile przypadków grypy jest w danych?
2. Która cecha najbardziej "wskazuje" na grypę? (porównaj P(cecha|TAK) vs P(cecha|NIE))
3. Stwórz wizualizację rozkładów cech dla każdej klasy

b) Predykcja dla 3 pacjentów
Dla każdego pacjenta oblicz prawdopodobieństwo grypy:
* Pacjent A: Gorączka=wysoka, Kaszel=TAK, Zmęczenie=duże
* Pacjent B: Gorączka=niska, Kaszel=NIE, Zmęczenie=małe
* Pacjent C: Gorączka=umiarkowana, Kaszel=TAK, Zmęczenie=umiarkowane

In [11]:
data = {
    'Gorączka': ['wysoka', 'wysoka', 'niska', 'wysoka', 'niska',
                 'wysoka', 'niska', 'wysoka', 'umiarkowana', 'wysoka',
                 'niska', 'umiarkowana', 'wysoka', 'niska', 'umiarkowana'],
    'Kaszel': ['TAK', 'TAK', 'NIE', 'TAK', 'NIE',
               'TAK', 'NIE', 'TAK', 'TAK', 'NIE',
               'NIE', 'TAK', 'TAK', 'NIE', 'TAK'],
    'Zmęczenie': ['duże', 'duże', 'małe', 'duże', 'małe',
                  'duże', 'małe', 'umiarkowane', 'duże', 'małe',
                  'małe', 'umiarkowane', 'duże', 'małe', 'duże'],
    'Grypa': ['TAK', 'TAK', 'NIE', 'TAK', 'NIE',
              'TAK', 'NIE', 'TAK', 'TAK', 'NIE',
              'NIE', 'NIE', 'TAK', 'NIE', 'TAK']
}

df_grypa = pd.DataFrame(data)

In [12]:
# a) Analiza danych
liczba_grypa = df_grypa['Grypa'].value_counts()
print("Liczba przypadków grypy:")
print(liczba_grypa)

# b) Predykcja dla pacjentów (A, B, C)
pacjenci = [
    {'Gorączka': 'wysoka', 'Kaszel': 'TAK', 'Zmęczenie': 'duże'},      # A
    {'Gorączka': 'niska', 'Kaszel': 'NIE', 'Zmęczenie': 'małe'},       # B
    {'Gorączka': 'umiarkowana', 'Kaszel': 'TAK', 'Zmęczenie': 'umiarkowane'} # C
]

p_grypa_tak = len(df_grypa[df_grypa['Grypa'] == 'TAK']) / len(df_grypa)
p_grypa_nie = len(df_grypa[df_grypa['Grypa'] == 'NIE']) / len(df_grypa)

def przewidz_grype(pacjent):
    # Dla TAK
    subset_tak = df_grypa[df_grypa['Grypa'] == 'TAK']
    prob_tak = 1.0
    for k, v in pacjent.items():
        prob_tak *= len(subset_tak[subset_tak[k] == v]) / len(subset_tak)
    post_tak = p_grypa_tak * prob_tak

    # Dla NIE
    subset_nie = df_grypa[df_grypa['Grypa'] == 'NIE']
    prob_nie = 1.0
    for k, v in pacjent.items():
        prob_nie *= len(subset_nie[subset_nie[k] == v]) / len(subset_nie)
    post_nie = p_grypa_nie * prob_nie

    # Normalizacja
    norm = post_tak + post_nie
    return post_tak / norm if norm > 0 else 0

print("\nWyniki diagnozy:")
imiona = ['Pacjent A', 'Pacjent B', 'Pacjent C']
for imie, p in zip(imiona, pacjenci):
    wynik = przewidz_grype(p)
    print(f"{imie}: P(Grypa=TAK) = {wynik:.4f} ({wynik*100:.1f}%)")

Liczba przypadków grypy:
Grypa
TAK    8
NIE    7
Name: count, dtype: int64

Wyniki diagnozy:
Pacjent A: P(Grypa=TAK) = 1.0000 (100.0%)
Pacjent B: P(Grypa=TAK) = 0.0000 (0.0%)
Pacjent C: P(Grypa=TAK) = 0.9245 (92.5%)
